In [3]:
from mia_models import train_target_model, load_shadow_data, train_shadow_models
from mia_models import prepare_attack_test_data, prep_validation_data, prep_attack_train_data
from mia_models import shokri_attack, prety_print_result
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import accuracy_score
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [ ]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [4]:
# original dataset with random index
dataset_name = 'avila'
train_size = 7000
attack_test_size = 2500
test_start = 7000
data = pd.read_csv('data/avila-tr.txt', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 10000, replace = False)
df_rest = pd.read_csv('data/avila-ts.txt', na_values=["?"], header=None)
shadow_dataset = df_rest.sample(n = 7000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members =  target_dataset.iloc[:train_size,:].sample(n = attack_test_size, replace = False)

In [5]:
# trian target model
per_class_sample=150
channel=0   
epochs=200
n_class = 12
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, n_class, train_size, test_start, is_synthetic)

2022-09-18 11:30:22.418702: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-18 11:30:22.418832: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-18 11:30:22.418908: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-BP5MTC3R): /proc/driver/nvidia/version does not exist
2022-09-18 11:30:22.419556: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



 Target Train acc :  99.92856979370117 Target Test acc :  98.66666793823242


In [6]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 7000

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  93.12244653701782 Shadow Test acc :  86.9523823261261
66/66 [==============================] - 0s 3ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  98.22449088096619 Shadow Test acc :  91.90475940704346
66/66 [==============================] - 0s 4ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  99.91836547851562 Shadow Test acc :  95.85714340209961
66/66 [==============================] - 0s 5ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  99.42857027053833 Shadow Test acc :  96.23809456825256
66/66 [==============================] - 0s 2ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  99.55102205276489 Shadow Test acc :  95.333331823349
66/66 [==============================] - 0s 3ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  99.93877410888672 Shadow Test acc :  96.71428799629211
66/66 [==============================] - 0s 3ms/

In [7]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

79/79 [==============================] - 0s 2ms/step
0
32/32 [==============================] - 0s 1ms/step
1
1/1 [==============================] - 0s 20ms/step
2
1/1 [==============================] - 0s 26ms/step
3
3/3 [==============================] - 0s 2ms/step
4
9/9 [==============================] - 0s 2ms/step
5
16/16 [==============================] - 0s 1ms/step
6
3/3 [==============================] - 0s 2ms/step
7
4/4 [==============================] - 0s 2ms/step
8
7/7 [==============================] - 0s 1ms/step
9
1/1 [==============================] - 0s 22ms/step
10
4/4 [==============================] - 0s 2ms/step
11
3/3 [==============================] - 0s 2ms/step
TP: 2498     FP: 2444     FN: 2     TN: 56
PPV: 0.5055
Advantage: 0.0216
Accuracy:  0.5108 Precision:  0.5054633751517604


In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#

In [12]:
# synthetic dataset
train_size = 7000
attack_test_size = 2500
test_start = 7000
target_dataset = pd.read_csv('data/avila_sds.csv', na_values=["?"], header=None)
df = pd.read_csv('data/avila-ts.txt', na_values=["?"], header=None)
shadow_dataset = df.sample(n = 7500, replace = False)
df_rest = df.loc[~df.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members = pd.read_csv('data/avila-tr.txt', na_values=["?"], header=None)
attack_test_members = attack_test_members.sample(n=attack_test_size, replace=False)

In [13]:
# trian target model
per_class_sample=150
channel=0   
epochs=200
n_class = 12
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, n_class, train_size, test_start, is_synthetic)


 Target Train acc :  99.9571442604065 Target Test acc :  99.15452003479004


In [10]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 7000

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  99.48979616165161 Shadow Test acc :  93.8095211982727
66/66 [==============================] - 0s 2ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  98.95918369293213 Shadow Test acc :  95.52381038665771
66/66 [==============================] - 0s 2ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  99.0612268447876 Shadow Test acc :  94.76190209388733
66/66 [==============================] - 0s 2ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  99.89795684814453 Shadow Test acc :  97.14285731315613
66/66 [==============================] - 0s 2ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  99.93877410888672 Shadow Test acc :  94.76190209388733
66/66 [==============================] - 0s 2ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  99.91836547851562 Shadow Test acc :  95.52381038665771
66/66 [==============================] - 0s 4ms

In [15]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

79/79 [==============================] - 0s 3ms/step
0
32/32 [==============================] - 0s 1ms/step
1
1/1 [==============================] - 0s 30ms/step
2
4/4 [==============================] - 0s 2ms/step
3
3/3 [==============================] - 0s 2ms/step
4
1/1 [==============================] - 0s 25ms/step
5
3/3 [==============================] - 0s 2ms/step
6
9/9 [==============================] - 0s 2ms/step
7
14/14 [==============================] - 0s 2ms/step
8
4/4 [==============================] - 0s 2ms/step
9
4/4 [==============================] - 0s 2ms/step
10
7/7 [==============================] - 0s 2ms/step
11
1/1 [==============================] - 0s 27ms/step
TP: 1189     FP: 1193     FN: 1311     TN: 1307
PPV: 0.4992
Advantage: -0.0016
Accuracy:  0.4992 Precision:  0.4991603694374475


In [ ]:
# --------------------------------------------Synthetic Data-------------------------------------------------------------#